# Création d'une carte interactive par catégories de lieux

*Document d'entrée* : le fichier au format JSON, comportant les lieux identifiés, leurs coordonnées géographiques et leurs types, créé lors de l'alignement des données (étape 3).

*Document de sortie* : carte au format HTML à ouvrir dans un navigateur web une fois téléchargée sur son ordinateur (en dehors du Google Drive).

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.



In [ ]:
#@markdown # Connecter le notebook à son compte Google Drive :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

!pip install geopy==1.17.0
from google.colab import drive
import urllib.parse, urllib.request
import os
import folium
from collections import Counter
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import csv
import re
import itertools
import matplotlib.pyplot as plt
from pandas.plotting import table

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

In [ ]:
#@markdown # Création d'une carte présentant les lieux par type

#@markdown Les coordonnées géographiques du fichier JSON sont récupérées pour intégrer directement les lieux sur la carte.

#@markdown La carte est par défaut centrée sur l'Europe. Il est possible de zoomer/dézoomer et de se déplacer pour accéder à l'endroit précis recherché.

#@markdown ####Indiquer le chemin absolu vers le dossier de travail sur le Google Drive :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/tutoriel_extraction_cartographie-main/'#@param {type:"string"}
os.chdir(chemin_vers_le_dossier_de_travail)

#@markdown ####Indiquer le nom du fichier JSON contenant les coordonnées géographiques :
nom_fichier_donnees_lieux = 'donnees lieux corpus guides.json'#@param {type:"string"}

#@markdown ####Indiquer le nom de la carte créée :
nom_carte = 'carte_categorisee.html'#@param {type:"string"}

liste_des_types = []
with open(nom_fichier_donnees_lieux, 'r') as po:
  contenu_json = json.load(po)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)

for ele in liste_des_types:
  if type(ele) == list:
    liste_des_types.remove(ele)
items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

### Création du fond de carte
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fg = folium.FeatureGroup(name="Tous")
map.add_child(fg)

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: 
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(contenu_json[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except (TypeError, ValueError) as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save(nom_carte)
map


In [ ]:
#@markdown # Ajout d'un fond de carte personnalisé

#@markdown L'ajout d'un fond de carte personnalisé est possible grâce au site web [Map Warper](https://mapwarper.net/). Les informations à ce sujet sont disponibles sur le tutoriel du [site du projet](https://baoia.huma-num.fr/contact/tutoriel-complet-de-lextraction-documentaire-a-la-cartographie/).
lien_tiles_pour_fond_carte = "https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png"#@param {type:"string"}
nom_carte_fond = 'carte_categorisee_fond.html'#@param {type:"string"}
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12, 
                 tiles=lien_tiles_pour_fond_carte, attr="None")

fg = folium.FeatureGroup(name="Tous")

map.add_child(fg)

liste_des_types = []
with open(nom_fichier_donnees_lieux, 'r') as po:
  contenu_json = json.load(po)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)

for ele in liste_des_types:
  if type(ele) == list:
    liste_des_types.remove(ele)
items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: ### boucle: pour chasue lieu dontenu dans "donnees"
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(contenu_json[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except (TypeError, ValueError) as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save(nom_carte_fond)
map
